VEAS - Daily counts of people of short, medium and long durations in the waste water collection area.

Preamble -- load essential libraries and define util functions


In [0]:
#!pip3 install pandas_gbq==0.8.0

  Found existing installation: pandas-gbq 0.11.0
    Uninstalling pandas-gbq-0.11.0:
      Successfully uninstalled pandas-gbq-0.11.0


In [0]:
# Load libraries 
from google.colab import auth
from google.cloud.bigquery import SchemaField
from google.cloud import bigquery
import time
from datetime import timedelta, date

In [2]:
#Authenticate on GCP
auth.authenticate_user()
print('Authenticated')

Authenticated


In [0]:
# Define a date range function for cleaner table suffix defines
from datetime import timedelta, date

def daterange(date1, date2):
    for n in range(int ((date2 - date1).days)+1):
        yield date1 + timedelta(n)

In [0]:
# Store results to table (adm_user_access)
project = 'telia-ddi-no-dev'
dataset = 'hannetorill'
table = 'delivery_veas_dwell_counts'
path  = dataset+"."+table

In [0]:
# Clear table if it already exists
from google.cloud.exceptions import NotFound

def table_exists(client, table_ref):
  try:
    client.get_table(table_ref)
    return True
  except NotFound:
    print("Table {} is not found.".format(table_id))
    print("Continuing...")
    return False

In [7]:
client = bigquery.Client(project=project)
table_ref = client.dataset(dataset).table(table)

if table_exists(client, table_ref):
  QUERY=f'DELETE FROM {path} where true'
  query_job = client.query(QUERY,location='EU') 
  query_job.result()  # Waits for the query to finish
  print('Cleared table {}'.format(table_ref.path))

Cleared table /projects/telia-ddi-no-dev/datasets/hannetorill/tables/delivery_veas_dwell_counts


In [0]:
def querystring(strdate):
  return f"""-- Define Area of Interest
with aoi as ( 
  SELECT
    m.*
    FROM `telia-ddi-no-prod.maps.administrative_level_4` as m
    inner join `telia-ddi-no-prod.maps.bydel_grunnkrets_mapping_2018` as b
      on m.admin_level_4_code = b.admin_level_4_code
    where b.bydel_name in ("Ullern", "Vestre Aker", "Frogner", "Nordre Aker", "Bjerke", "St. Hanshaugen", "Sagene")
  union all
  SELECT * FROM `telia-ddi-no-prod.maps.administrative_level_4` as n
    where n.admin_level_2 in ("Bærum", "Asker")
    or (n.admin_level_2 in ("Nesodden") and n.admin_level_3 in ("Bjørnemyr"))
    or (n.admin_level_2 in ("Røyken") and n.admin_level_3 in ("Østbygda slemmestad"))
    or (n.admin_level_2 in ("Røyken") and n.admin_level_4 in ("Hallenskog", "Wang", "Gleinåsen/Katrineåsen", "Heggum"))
    or (n.admin_level_2 in ("Nesodden") and n.admin_level_4 in ("Ellingstad"))
 )
 -- Extrapolate from market share
 , ex_customers AS (
  SELECT
    (1 / p_observation) * m_cluster_adjuster * m_device_reducer AS m_customers_morning
  FROM `telia-ddi-no-prod.extrapolation.signal_probabilities_day_part_{strdate}`
  WHERE day_part="Morning"
)
, ex_identifier AS (
  SELECT
    identifier, observed_market_share
  FROM `telia-ddi-no-prod.mobility_signals.areas_of_habit_{strdate}` h
  INNER JOIN `telia-ddi-no-prod.maps.administrative_level_4` ma ON ST_CONTAINS(ma.geo, h.point)
  INNER JOIN `telia-ddi-no-prod.extrapolation.administrative_level_3_baseline_market_share` ms ON ms.admin_level_3_code = ma.admin_level_3_code
  WHERE h.aoh_classification = 'HOME'
)
-- Get the data
, data as (
  SELECT
    m.admin_level_4_code AS area_code
    , m.admin_level_2 as area
    , ma.admin_level_4_code AS aoh_area_code
    , ma.admin_level_2 as aoh_area
    , h.aoh_classification
    , c.identifier
    , TIMESTAMP_TRUNC(c.start_time, DAY) AS utc_day
    , c.duration
  -- get the high quality dwells in the area of interest
  FROM `telia-ddi-no-prod.mobility_signals.dwells_{strdate}` c
  INNER JOIN `telia-ddi-no-prod.mobility_signals.identifiers_{strdate}` t ON c.identifier = t.identifier
  INNER JOIN aoi m ON c.geo_id_admin_level_4 = m.geo_id
  -- find the aoh for the signals
  INNER JOIN `telia-ddi-no-prod.mobility_signals.areas_of_habit_{strdate}` h ON c.identifier = h.identifier
  INNER JOIN `telia-ddi-no-prod.maps.administrative_level_4` ma ON ST_CONTAINS(ma.geo, h.point)
  WHERE t.rating NOT IN ('C')
    and duration >= 5*60*60
)
, visit_durations as (
  select d.*
  , case
      when (d.duration < 8*60*60) then "SHORT"
      when d.duration >= 12*60*60 then "LONG"
      else "MEDIUM" end as dur_type
  , (m_customers_morning/i.observed_market_share) as people
  from data d, ex_customers
  LEFT JOIN ex_identifier i
  ON i.identifier = d.identifier
)
, short_durs as (
  select area, date(utc_day) as dato, people
  from visit_durations v, ex_customers
  where dur_type = "SHORT"
)
, medium_durs as (
  select  area, date(utc_day) as dato, people
  from visit_durations v, ex_customers
  where dur_type = "MEDIUM"
)
, long_durs as (
  select area, date(utc_day) as dato, people
  from visit_durations v, ex_customers
  where dur_type = "LONG"
)
select 
  s.area, s.dato, s.people_short, m.people_medium, l.people_long
from (select DENSE_RANK() OVER(ORDER BY dato, area) AS id
    , area, dato, CAST(ROUND(sum(people)) AS INT64) as people_short
  from short_durs group by dato, area) s
  inner join ( select DENSE_RANK() OVER(ORDER BY dato, area) AS id
    , area, dato, CAST(ROUND(sum(people)) AS INT64) as people_medium
    from medium_durs group by dato, area ) m on s.id = m.id
  inner join ( select DENSE_RANK() OVER(ORDER BY dato, area) AS id
    , area, dato, CAST(ROUND(sum(people)) AS INT64) as people_long
    from long_durs group by dato, area ) l on s.id = l.id
"""

In [18]:
start_date = date(2019, 11, 4)
end_date = date(2019, 11, 30)

for single_date in daterange(start_date, end_date):
  print (single_date.strftime("%Y%m%d"))
  strdate = single_date.strftime("%Y%m%d")+""
  time.sleep(2)
  client = bigquery.Client(project=project)
  QUERY  = querystring(strdate)
  query_job = client.query(QUERY, location="EU") 
  table_ref = client.dataset(dataset).table(table)
  e = query_job.to_dataframe()
  print(e)
   
  e.to_gbq(path, project, None, 
           False, 'append', False, None, "EU", True, None, None, None)

20191104
       area        dato  people_short  people_medium  people_long
0     Asker  2019-11-04         86902          49452        20314
1     Bærum  2019-11-04        183882         104710        40514
2  Nesodden  2019-11-04          3993           2040         1049
3      Oslo  2019-11-04        547703         311046        97406
4    Røyken  2019-11-04         15522           8172         5069


1it [00:04,  4.11s/it]


20191105
       area        dato  people_short  people_medium  people_long
0     Asker  2019-11-05         85900          49296        20295
1     Bærum  2019-11-05        182211         103381        41049
2  Nesodden  2019-11-05          4204           1860         1093
3      Oslo  2019-11-05        554161         309479        95900
4    Røyken  2019-11-05         14799           8305         4860


1it [00:03,  3.71s/it]


20191106
       area        dato  people_short  people_medium  people_long
0     Asker  2019-11-06         85003          48231        20769
1     Bærum  2019-11-06        182181         101400        41271
2  Nesodden  2019-11-06          3571           2010         1065
3      Oslo  2019-11-06        546472         309858        98378
4    Røyken  2019-11-06         14781           7866         5022


...

1it [00:04,  4.15s/it]


20191129
       area        dato  people_short  people_medium  people_long
0     Asker  2019-11-29         80052          45441        21021
1     Bærum  2019-11-29        172056          95726        42943
2  Nesodden  2019-11-29          3266           1738         1043
3      Oslo  2019-11-29        522039         278021        97223
4    Røyken  2019-11-29         13893           8160         4945


1it [00:03,  3.36s/it]


20191130
       area        dato  people_short  people_medium  people_long
0     Asker  2019-11-30         24100          29993        26638
1     Bærum  2019-11-30         48843          60950        54461
2  Nesodden  2019-11-30          1020           1217         1372
3      Oslo  2019-11-30        133246         161529       129660
4    Røyken  2019-11-30          4821           5559         5845


1it [00:04,  4.41s/it]


2019-11-01
